In [ ]:
import numpy as np
import pandas as pd
import bottleneck as bn
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.sparse as sparse
import bottleneck as bn
import copy
import torch

In [ ]:
def Recall_at_k_batch(X_pred, heldout_batch, k=100):
    batch_users = X_pred.shape[0]
    idx = bn.argpartition(-X_pred, k, axis=1)  # indices of the top-k items (not in order).
    X_pred_binary = np.zeros_like(X_pred, dtype=bool)  # [B, I]
    X_pred_binary[np.arange(batch_users)[:, np.newaxis], idx[:, :k]] = True
    X_true_binary = (heldout_batch > 0)  # .toarray() #  [B, I]
    tmp = (np.logical_and(X_true_binary, X_pred_binary).sum(axis=1)).astype(
        np.float32)
    recall = tmp / np.minimum(k, X_true_binary.sum(axis=1))
    return recall

def NDCG_binary_at_k_batch(X_pred, heldout_batch, k=100):
    """
    Normalized Discounted Cumulative Gain@k for binary relevance
    ASSUMPTIONS: all the 0's in heldout_data indicate 0 relevance
    """
    batch_users = X_pred.shape[0]  # batch_size
    idx_topk_part = bn.argpartition(-X_pred, k, axis=1)
    topk_part = X_pred[np.arange(batch_users)[:, np.newaxis],
                       idx_topk_part[:, :k]]
    idx_part = np.argsort(-topk_part, axis=1)

    idx_topk = idx_topk_part[np.arange(batch_users)[:, np.newaxis], idx_part]

    tp = 1. / np.log2(np.arange(2, k + 2))

    DCG = (heldout_batch[np.arange(batch_users)[:, np.newaxis],
                         idx_topk] * tp).sum(axis=1)

    IDCG = np.array([(tp[:min(n, k)]).sum()
                     for n in np.count_nonzero(heldout_batch, axis=1)])
    return DCG / IDCG


In [ ]:
def load_prediction(fname):
    """ Load model output
        We assume predictions & targets (ground_truth) are aligned
        Return - a list of model outputs"""
    try:
        preds = np.load(fname, allow_pickle=True)['preds']
        if len(preds.shape) ==1:
            return np.squeeze(np.concatenate(preds, axis=0))
        else:
            return np.squeeze(preds)
    except:
        try:
            preds = np.load(fname, allow_pickle=True)['arr_0']
            if len(preds.shape) ==1:
                return np.squeeze(np.concatenate(preds, axis=0))
            else:
                return np.squeeze(preds)            
        except:
            return None
    

def load_targets(dataset, mode='test', num_tr=5):
    """ Load ground truth val/test user-item matrix along with user activity (# interacted items in
    the fold-in set)"""
    df_ui = pd.read_csv('~/few-shot-rec/data/{}/popular_train.csv'.format(dataset))  # read train set.
    n_items = df_ui['item'].max() + 1
    n_users = df_ui['user'].max() + 1
    print ("# unique users", len(df_ui.user.unique()))
    
    all_user_activity = df_ui.groupby('user').size().reset_index(name='counts') #.values.astype(np.float32)
        
    print ("n_users", n_users)
    
    df_ui_tr_te = pd.read_csv('~/few-shot-rec/data/{}/{}.csv'.format(dataset, mode))
    df_ui_tr_te = df_ui_tr_te[['item', 'user_0']]
    df_ui_tr_te = df_ui_tr_te.rename({'user_0': 'user'}, axis=1)
    
    df_ui_eval_tr = df_ui_tr_te.groupby('item').apply(lambda x: x[:num_tr])
    df_ui_eval_te = df_ui_tr_te[~df_ui_tr_te.index.isin(df_ui_eval_tr.index.get_level_values(1))]
        
    print (len(df_ui_eval_tr), len(df_ui_eval_te), len(df_ui_tr_te))
    print (df_ui_eval_te.columns, df_ui_eval_tr.columns)

    start_idx, end_idx = df_ui_eval_te['item'].min(), df_ui_eval_te['item'].max()
    n_few_shot_items = end_idx - start_idx + 1
    
    rows_ui, cols_ui = df_ui_eval_te['user'], df_ui_eval_te['item'] - start_idx
    rows_ui_tr, cols_ui_tr = df_ui_eval_tr['user'], df_ui_eval_tr['item'] - start_idx
    
    
    
    fold_in = sparse.csr_matrix((np.ones_like(rows_ui_tr), (rows_ui_tr, cols_ui_tr)), 
                                     dtype='float32', shape=(n_users, n_few_shot_items))
    
    # [# eval users x items] sparse matrix.
    data_ui_test = sparse.csr_matrix((np.ones_like(rows_ui), (rows_ui, cols_ui)), 
                                     dtype='float32', shape=(n_users, n_few_shot_items))
    
    test_users = torch.arange(n_users)[~torch.all(torch.from_numpy(data_ui_test.toarray()) == 0, dim=1)].numpy()
    print ("# test_users", len(test_users))
        
    # test_users = df_ui_eval_te['user'].unique()
    n_test_users = len(test_users)
    
    test_users_activity = np.zeros(n_test_users)
    for idx, u in enumerate(test_users):
        try:
            test_users_activity[idx]  = all_user_activity[all_user_activity['user'] == u]['counts'].values[0]
        except:
            test_users_activity[idx] = 0
            
    fold_in = fold_in[test_users, :]
    data_ui_test = data_ui_test[test_users, :]  # n_test_users, n_few_shot_items
    return fold_in.toarray(), data_ui_test.toarray(), test_users_activity

def load_targets_all(dataset, num_tr=5, mode='popular'):
    """ Load ground truth val/test user-item matrix along with user activity (# interacted items in
    the fold-in set)"""
    df_ui = pd.read_csv('~/few-shot-rec/data/{}/popular_train.csv'.format(dataset))  # read train set.
    n_items = df_ui['item'].max() + 1
    n_users = df_ui['user'].max() + 1
    print ("# unique users", len(df_ui.user.unique()))
    
    all_user_activity = df_ui.groupby('user').size().reset_index(name='counts') #.values.astype(np.float32)
        
    print ("n_users", n_users)
    if mode == "popular":
        df_ui_eval_tr = df_ui
        df_ui_tr_te = pd.read_csv('~/few-shot-rec/data/{}/popular_test.csv'.format(dataset, mode))
        df_ui_eval_te = df_ui_tr_te
        df_popular = pd.DataFrame()
        df_popular = df_popular.append(df_ui_eval_tr)
        df_popular = df_popular.append(df_ui_eval_te)
        all_item_activity = df_popular.groupby('item').size().reset_index(name='counts')
    else:
        df_ui_tr_te = pd.read_csv('~/few-shot-rec/data/{}/test.csv'.format(dataset))
        df_ui_tr_te = df_ui_tr_te[['item', 'user_0']]
        df_ui_tr_te = df_ui_tr_te.rename({'user_0': 'user'}, axis=1)    
#         df_val_tr_te = pd.read_csv('~/few-shot-rec/data/{}/val.csv'.format(dataset))
#         df_val_tr_te = df_val_tr_te[['item', 'user_0']]
#         df_val_tr_te = df_val_tr_te.rename({'user_0': 'user'}, axis=1)
#         df_ui_tr_te = df_ui_tr_te.append(df_val_tr_te)
        
        all_item_activity = df_ui_tr_te.groupby('item').size().reset_index(name='counts')        
        df_ui_eval_tr = df_ui_tr_te.groupby('item').apply(lambda x: x[:int(0.7 * len(x))])
        # df_ui_eval_tr = df_ui_eval_tr.append(df_val_tr_te.groupby('item').apply(lambda x: x[:num_tr]))                
        df_ui_eval_te = df_ui_tr_te[~df_ui_tr_te.index.isin(df_ui_eval_tr.index.get_level_values(1))]

    print (len(df_ui_eval_tr), len(df_ui_eval_te), len(df_ui_tr_te))
    print (df_ui_eval_te.columns, df_ui_eval_tr.columns)

    start_idx, end_idx = df_ui_eval_te['item'].min(), df_ui_eval_te['item'].max()
    n_few_shot_items = end_idx - start_idx + 1
    
    rows_ui, cols_ui = df_ui_eval_te['user'], df_ui_eval_te['item'] - start_idx
    rows_ui_tr, cols_ui_tr = df_ui_eval_tr['user'], df_ui_eval_tr['item'] - start_idx
    
        
    fold_in = sparse.csr_matrix((np.ones_like(rows_ui_tr), (rows_ui_tr, cols_ui_tr)), 
                                     dtype='float32', shape=(n_users, n_few_shot_items))
    
    # [# eval users x items] sparse matrix.
    data_ui_test = sparse.csr_matrix((np.ones_like(rows_ui), (rows_ui, cols_ui)), 
                                     dtype='float32', shape=(n_users, n_few_shot_items))
    
    test_users = torch.arange(n_users)[~torch.all(torch.from_numpy(data_ui_test.toarray()) == 0, dim=1)].numpy()
    print ("# test_users", len(test_users))
        
    # test_users = df_ui_eval_te['user'].unique()
    n_test_users = len(test_users)
    
    items_activity = np.zeros(n_few_shot_items)
    for idx, item in enumerate(range(start_idx, end_idx+1)):
        try:
            items_activity[idx]  = all_item_activity[all_item_activity['item'] == item]['counts'].values[0]
        except:
            items_activity[idx] = 0
            
    test_users_activity = np.zeros(n_test_users)
    for idx, u in enumerate(test_users):
        try:
            test_users_activity[idx] = all_user_activity[all_user_activity['user'] == u]['counts'].values[0]
        except:
            test_users_activity[idx] = 0
            
    fold_in = fold_in[test_users, :]
    data_ui_test = data_ui_test[test_users, :]  # n_test_users, n_few_shot_items
    return fold_in.toarray(), data_ui_test.toarray(), test_users_activity, test_users, items_activity

In [ ]:
def load_targets_joint(dataset):
    """ Load ground truth val/test user-item matrix along with user activity (# interacted items in
    the fold-in set)"""
    df_popular_train = pd.read_csv('~/few-shot-rec/data/{}/popular_train.csv'.format(dataset))  # read train set.
    n_popular = df_popular_train['item'].max() + 1
    n_users = df_popular_train['user'].max() + 1
    print ("max users", n_users)
    print ("# unique users", len(df_popular_train.user.unique()))
    
    all_user_activity = df_popular_train.groupby('user').size().reset_index(name='counts') #.values.astype(np.float32)
    
    df_all = pd.DataFrame()
    
    df_popular_test = pd.read_csv('~/few-shot-rec/data/{}/popular_test.csv'.format(dataset))
    df_popular_eval = pd.read_csv('~/few-shot-rec/data/{}/popular_val.csv'.format(dataset))
    
    df_popular = pd.DataFrame()
    df_all = df_all.append(df_popular_train)
    df_all = df_all.append(df_popular_test)
        
    df_fs_test = pd.read_csv('~/few-shot-rec/data/{}/test.csv'.format(dataset))
    df_all = df_all.append(df_fs_test)
    
    df_fs_test = df_fs_test[['item', 'user_0']]
    df_fs_test = df_fs_test.rename({'user_0': 'user'}, axis=1)  
    
    df_fs_train = df_fs_test.groupby('item').apply(lambda x: x[:int(0.7 * len(x))])
    df_fs_test = df_fs_test[~df_fs_test.index.isin(df_fs_train.index.get_level_values(1))]
    
    ## GETTING FEWSHOT TARGET
    start_idx, end_idx = df_fs_test['item'].min(), df_fs_test['item'].max()
    n_few_shot_items = end_idx - start_idx + 1
    
    rows_ui, cols_ui = df_fs_test['user'], df_fs_test['item'] - start_idx
    
    # [# eval users x items] sparse matrix.
    data_fs_test = sparse.csr_matrix((np.ones_like(rows_ui), (rows_ui, cols_ui)), 
                                     dtype='float32', shape=(n_users, n_few_shot_items))
    
    ## GETTING POPULAR TARGET
    rows_te, cols_te = df_popular_test['user'], df_popular_test['item']
    
    ui_data_te = sparse.csr_matrix((np.ones_like(rows_te),
                                    (rows_te, cols_te)), dtype='float32',
                                   shape=(n_users, n_popular))
    
    all_target = sparse.hstack([ui_data_te, data_fs_test])    
    all_item_activity = df_all.groupby('item').size().reset_index(name='counts')
    items_activity = np.zeros(n_popular + n_few_shot_items)
    
    for idx, item in enumerate(range(0, n_popular+1)):
        try:
            items_activity[idx]  = all_item_activity[all_item_activity['item'] == item]['counts'].values[0]
        except:
            items_activity[idx] = 0
    
    
    for idx, item in enumerate(range(start_idx, end_idx+1)):
        try:
            items_activity[n_popular+idx]  = all_item_activity[all_item_activity['item'] == item]['counts'].values[0]
        except:
            items_activity[n_popular+idx] = 0
    test_users = np.arange(0, n_users)

    if dataset == 'yelp_AZ0.3_processed_item':
        test_users = np.array(all_user_activity[all_user_activity.counts > 5].user)
        
    if dataset == 'gowalla0.3_processed_item':
        test_users = np.array(all_user_activity[all_user_activity.counts > 10].user)
        
    if dataset == 'weeplaces0.3_processed_item':
        test_users = np.array(all_user_activity[all_user_activity.counts > 15].user)

    print ("# test users", len(test_users))
    # test_users = 
    
#     test_users = torch.arange(n_users)[~torch.all(torch.from_numpy(data_ui_test.toarray()) == 0, dim=1)].numpy()
#     print ("# test_users", len(test_users))
        
#     # test_users = df_ui_eval_te['user'].unique()
#     n_test_users = len(test_users)
    
#     items_activity = np.zeros(n_few_shot_items)
#     for idx, item in enumerate(range(start_idx, end_idx+1)):
#         try:
#             items_activity[idx]  = all_item_activity[all_item_activity['item'] == item]['counts'].values[0]
#         except:
#             items_activity[idx] = 0
            
#     test_users_activity = np.zeros(n_test_users)
#     for idx, u in enumerate(test_users):
#         try:
#             test_users_activity[idx] = all_user_activity[all_user_activity['user'] == u]['counts'].values[0]
#         except:
#             test_users_activity[idx] = 0
            
#     fold_in = fold_in[test_users, :]
#     data_ui_test = data_ui_test[test_users, :]  # n_test_users, n_few_shot_items
    return all_target.toarray(), items_activity, test_users

# all_target, items_activity, test_users = load_targets_joint('{}_processed_item'.format('gowalla0.3'))
#print ("all_target", all_target.shape, "items_activity", items_activity.shape, "test_users", test_users.shape)

In [ ]:
 #len(all_user_activity[all_user_activity.counts > 10])

In [ ]:
def Recall_at_k_batch_torch(X_pred, heldout_batch, k=100):
    batch_users = X_pred.shape[0]
    _, topk_indices = torch.topk(X_pred, k, dim=1, sorted=False)  # [B, K]
    X_pred_binary = torch.zeros_like(X_pred)
    if torch.cuda.is_available():
        X_pred_binary = X_pred_binary.cuda()
    X_pred_binary[torch.arange(batch_users).unsqueeze(1), topk_indices] = 1
    X_true_binary = (heldout_batch > 0).float()  # .toarray() #  [B, I]
    k_tensor = torch.Tensor([k])
    if torch.cuda.is_available():
        X_true_binary = X_true_binary.cuda()
        k_tensor = k_tensor.cuda()
    tmp = (X_true_binary * X_pred_binary).sum(dim=1).float()
    recall = tmp / torch.min(k_tensor, X_true_binary.sum(dim=1).float())
    return recall




#all_target = torch.tensor(all_target)
#vaepred = torch.tensor(load_prediction('/data/prediction/our_model/{}bpr_memory_few_shot_test_preds_new.npz'.format('weeplaces0.3')))
#vaepred = torch.tensor(load_prediction('/data/prediction/our_model/{}vae_memory_few_shot_test_preds_new.npz'.format('weeplaces0.3')))

#print(vaepred.shape)
#unique_users = torch.arange(vaepred.shape[0])[~torch.all(all_target == 0, dim=1)].numpy()

#print(torch.mean(Recall_at_k_batch_torch(vaepred[unique_users], all_target[unique_users], k=50)))



In [ ]:

def pandas_explode(df, column_to_explode, cols):
    # Create a list of new observations
    new_observations = list()
    # Iterate through existing observations
    for row in df.to_dict(orient='records'):
        # Take out the exploding iterable
        explode_values = row[column_to_explode]
        del row[column_to_explode]
        # Create a new observation for every entry in the exploding iterable & add all of the other columns
        
        print(len(explode_values))
        for idx, explode_value in enumerate(explode_values):
            # Deep copy existing observation
            new_observation = copy.deepcopy(row)
            # Add one (newly flattened) value from exploding iterable
            new_observation[column_to_explode] = explode_value
            for c_name in cols:
                print(c_name)
                print(idx)
                print(cols[c_name])
                new_observation[c_name] = cols[c_name][idx]
            # new_observation['group_size'] = g_sizes[idx]
            # Add to the list of new observations
            new_observations.append(new_observation)

    # Create a DataFrame
    return_df = pd.DataFrame(new_observations)

    # Return
    return return_df



def get_df(dataset, models_paths, metric, K, mode='test'):
    df = pd.DataFrame(columns=['dataset', 'variant', 'metric', 'K', 'score'])
    fold_in, targets, user_activity = load_targets(dataset, mode=mode, num_tr=5)
    rows_list = []
    if metric == 'NDCG':
        metric_fn = NDCG_binary_at_k_batch
    else:    
        metric_fn = Recall_at_k_batch
    
    for model, path in models_paths.items():
        preds = load_prediction(path)
        if preds is None:
            continue
        if model == 'Ours':
            preds = preds.transpose()            
        print ("preds", preds.shape, "targets", targets.shape, "activity", user_activity.shape)
        # preds[fold_in.nonzero()] = -np.inf
        score = metric_fn(preds, targets, K)
        row_dict = {'dataset': dataset, 'variant': model, 'metric': metric, 'K' : K, 'score': score}
        rows_list.append(row_dict)
    df = pd.DataFrame(rows_list)
    df = pandas_explode(df, 'score', cols={'user_activity': user_activity})
    return df


def get_df_all(dataset, model_paths, metric, K, mode='popular'):
    df = pd.DataFrame(columns=['dataset', 'variant', 'metric', 'K', 'score'])
    fold_in_popular, targets_popular, user_activity_popular,popular_test_user = load_targets_all(dataset, num_tr=5, mode='popular')
    fold_in_fs, targets_fs, user_activity_fs, fs_test_user = load_targets_all(dataset, num_tr=5, mode="fewshot")
    rows_list_popular = []
    rows_list_fs = []
    if metric == 'NDCG':
        metric_fn = NDCG_binary_at_k_batch
    else:    
        metric_fn = Recall_at_k_batch
    for model, path in model_paths.items():
        preds = load_prediction(path)
        print(preds)
        if preds is None:
            continue
        if model == 'Ours':
            preds = preds.transpose()            
        # print ("preds", preds.shape, "targets", targets.shape, "activity", user_activity.shape)
        # preds[fold_in.nonzero()] = -np.inf
        score_popular = metric_fn(preds[:, :targets_popular.shape[1]], targets_popular, K)
        score_fs = metric_fn(preds[:, targets_fs.shape[1]:], targets_fs, K)
        row_dict = {'dataset': dataset, 'variant': model, 'metric': metric, 'K' : K, 'score': score_popular}
        row_dict2 = {'dataset': dataset, 'variant': model, 'metric': metric, 'K' : K, 'score': score_fs}
        rows_list_popular.append(row_dict)
        rows_list_fs.append(row_dict2)
    df_popular = pd.DataFrame(rows_list_popular)
    df_popular = pandas_explode(df_popular, 'score', cols={'user_activity': user_activity_popular})
    df_fs = pd.DataFrame(rows_list_fs)
    df_fs = pandas_explode(df_fs, 'score', cols={'user_activity': user_activity_fs})
    return df_popular, df_fs

In [ ]:
# Helper functions to create bins based on a specified value (say user activity).

def cut(df, n_bins, key, range_min=None, range_max=None):
    if range_min is not None:
        assert range_max is not None
        ranges = np.arange(range_min, range_max + 0.1, (range_max - range_min) / n_bins)
        # ranges[-1] = float(df[key].max())
        print(ranges)
        bins = pd.cut(df[key], ranges)
        # bins = pd.qcut(df[key], n_bins)#, duplicates='drop')
    else:
        bins = pd.cut(df[key], n_bins)
        bins = pd.qcut(df[key], n_bins)#, duplicates='drop')
    new_df = df.copy()
    new_df['bin'] = bins
    return new_df

def bin_data(df, key, n_bins, metric, range_min=None, range_max=None):
    if range_min is not None:
        assert range_max is not None
        all_data_binned = cut(df, n_bins, key, range_min, range_max)
    else:
        all_data_binned = cut(df, n_bins, key)
    return all_data_binned

In [ ]:
import time
# dataset = 'gowalla0.5'
#dataset = 'weeplaces'
# dataset = 'weeplaces0.3'
# dataset = 'gowalla0.3'
# dataset = 'epinions0.3'
dataset = 'yelp_AZ0.3'
datasets = ['{}_processed_item'.format(dataset)]
# Static best model output paths for each dataset.
models_paths = {}
models_paths['{}_processed_item'.format(dataset)] = {
    'BPR' : '/data/prediction/BPR/{}_all_preds_new.npz'.format(dataset),
    'VAE-CF' : "/data/prediction/VAE-CF/{}_all_preds_new.npz".format(dataset),
    # 'DAE' : "/data/prediction/DAE/{}_5_all_preds.npz".format(dataset),
    # 'DAE_new' : "/data/prediction/DAE/{}_5_new_all_preds.npz".format(dataset),
    # 'NCF' : '/data/prediction/NCF/{}_5_all_preds.npz'.format(dataset)
    'Ours_VAE': '/data/prediction/our_model/{}vae_memory_few_shot_test_preds_new.npz'.format(dataset),
    'Ours_BPR': '/data/prediction/our_model/{}bpr_memory_few_shot_test_preds_new.npz'.format(dataset),
    "Meta_Rec": '/data/prediction/meta_learning/{}vae_memory_few_shot_test_preds_new.npz'.format(dataset)
}
print(models_paths)

# Define best model paths here for each dataset (has to be hardcoded).

# df = pd.DataFrame()
# for ds in datasets:
#     print (ds)
#     df_ds = get_df(ds, models_paths[ds],'Recall', 50)
#     df = df.append(df_ds)

dataset = datasets[0]

In [ ]:
def get_item_recall_outputs(targets, preds, test_users, item_activity):
    targets_expanded, preds_expanded, users_expanded, items_expanded, item_activity_expanded = [], [], [], [], []
    for u, pred, target in zip(test_users, preds, targets):
        items = target.nonzero()[0]
        for item in items:
            arr = np.zeros(target.shape[0])
            arr[item] = 1
            targets_expanded.append(arr)
            # targets_expanded.append(target)
            preds_expanded.append(pred)
            users_expanded.append(u)
            items_expanded.append(item)
            item_activity_expanded.append(item_activity[item])
    targets_expanded, preds_expanded, users_expanded, items_expanded, item_activity_expanded = \
    np.array(targets_expanded), np.array(preds_expanded), np.array(users_expanded), \
    np.array(items_expanded), np.array(item_activity_expanded)
    
    return targets_expanded, preds_expanded, users_expanded, items_expanded, item_activity_expanded


In [ ]:
def get_df_item_recall(dataset, model_paths, metric, K, mode='popular'):
    # fold_in_popular, targets_popular, user_activity_popular,popular_test_user, item_activity_popular = load_targets_all(dataset, num_tr=5, mode='popular')
            
    # fold_in_fs, targets_fs, user_activity_fs, fs_test_user, item_activity_fs = load_targets_all(dataset, num_tr=5, mode="fewshot")
    
    targets, item_activity, test_users = load_targets_joint(dataset)  # TODO: get item_activity
    
    # print (user_activity_popular.shape, item_activity_popular.shape)
    # assuming that masking has already been done.
    # df_popular = pd.DataFrame(columns=['dataset', 'variant', 'metric', 'K', 'score'])

    # df_popular = pd.DataFrame()
    # df_fs = pd.DataFrame()

    df = pd.DataFrame()
    # rows_list_popular = []
    # rows_list_fs = []
    rows_list = []
    for model, path in model_paths.items():
        print (model, path)
        preds = load_prediction(path)
        print ("preds", preds.shape)
        if preds is None:
            continue
        if model == 'Ours':
            preds = preds.transpose()
            
        # preds_popular = preds[popular_test_user, :targets_popular.shape[1]]
        # preds_fs = preds[fs_test_user, targets_popular.shape[1]:]
        # print ("preds_popular", preds_popular.shape, "targets_popular", targets_popular.shape)
        # print ("preds_fs", preds_fs.shape, "targets_fs", targets_fs.shape)
        print ("preds", preds.shape, "targets", targets.shape)
                        
#         targets_expanded_popular, preds_expanded_popular, users_expanded_popular, items_expanded_popular, \
#         item_activity_expanded_popular = get_item_recall_outputs(targets_popular, preds_popular, popular_test_user, 
#                                                                 item_activity_popular)

#         targets_expanded_fs, preds_expanded_fs, users_expanded_fs, items_expanded_fs, item_activity_expanded_fs = \
#         get_item_recall_outputs(targets_fs, preds_fs, fs_test_user, item_activity_fs)
        if metric == 'NDCG':
            metric_fn = NDCG_binary_at_k_batch
        else:    
            metric_fn = Recall_at_k_batch
        
        df_model = pd.DataFrame()
        
        n_test_users = len(test_users)
        for user_batch in np.array_split(test_users, 500):
            batch_targets_expanded, batch_preds_expanded, batch_users_expanded, batch_items_expanded, \
            batch_item_activity_expanded = get_item_recall_outputs(targets[user_batch, :], 
                                                                   preds[user_batch, :], 
                                                                   user_batch, item_activity)            
            
#             targets_expanded, preds_expanded, users_expanded, items_expanded, item_activity_expanded = \
#             get_item_recall_outputs(targets, preds, test_users, item_activity)   
        

            row_dict = {'variant': model, 'user': batch_users_expanded, 'item': batch_items_expanded, 
                           'item_activity': batch_item_activity_expanded,
                           'metric': metric, 'K' : K, 
                            'score': metric_fn(batch_preds_expanded, batch_targets_expanded, K)}
        
#             row_dict = {'variant': model, 'user': users_expanded, 'item': items_expanded, 
#                            'item_activity': item_activity_expanded,
#                            'metric': metric, 'K' : K, 
#                             'score': metric_fn(preds_expanded, targets_expanded, K)}

            df_model = df_model.append(pd.DataFrame(row_dict))
        
        # df_model = pd.DataFrame(row_dict).groupby(['variant', 'item', 'metric', 'K']).mean().reset_index()
        df_model = df_model.groupby(['variant', 'item', 'metric', 'K']).mean().reset_index()
        df = df.append(df_model)
        
#         row_dict_popular = {'variant': model,  'user': users_expanded_popular, 'item': items_expanded_popular, 
#                             'item_activity': item_activity_expanded_popular,
#                             'metric': metric, 'K' : K, 
#                             'score': metric_fn(preds_expanded_popular, targets_expanded_popular, K)}
        
#         row_dict_fs = {'variant': model, 'user': users_expanded_fs, 'item': items_expanded_fs, 
#                        'item_activity': item_activity_expanded_fs,
#                        'metric': metric, 'K' : K, 
#                         'score': metric_fn(preds_expanded_fs, targets_expanded_fs, K)}
                        
#         df_popular_model = pd.DataFrame(row_dict_popular).groupby(['variant', 'item', 'metric', 'K']).mean().reset_index()
#         df_fs_model = pd.DataFrame(row_dict_fs).groupby(['variant', 'item', 'metric', 'K']).mean().reset_index()
                                
#         df_popular = df_popular.append(df_popular_model)
#         df_fs = df_fs.append(df_fs_model)
        

    return df
#models_paths
dataset = datasets[0]


In [ ]:
dataset

In [ ]:
df_yelp = get_df_item_recall(dataset, models_paths[dataset], 'Recall', 50)


In [ ]:
dataset = 'weeplaces0.3'
models_paths = {}
models_paths['{}_processed_item'.format(dataset)] = {
    'BPR' : '/data/prediction/BPR/{}_all_preds_new.npz'.format(dataset),
    'VAE-CF' : "/data/prediction/VAE-CF/{}_all_preds_new.npz".format(dataset),
    'Ours_VAE': '/data/prediction/our_model/{}vae_memory_few_shot_test_preds_new.npz'.format(dataset),
    'Ours_BPR': '/data/prediction/our_model/{}bpr_memory_few_shot_test_preds_new.npz'.format(dataset)
}
df_weeplaces = get_df_item_recall('weeplaces0.3_processed_item', models_paths['weeplaces0.3_processed_item'], 'Recall', 50)


In [ ]:
n_bins= 10
df_weeplaces['dataset'] = 'weeplaces'

data_binned = pd.DataFrame()
ds = df_weeplaces[(df_weeplaces.dataset == 'weeplaces') & (df_weeplaces.item_activity > 0)  ]

binned_ds = bin_data(ds, 'item_activity', n_bins, 'score')
data = binned_ds
idx = binned_ds.groupby('bin').size().reset_index()['bin'].reset_index()
binned_ds = binned_ds.merge(idx, on='bin', how='inner')
data_binned = data_binned.append(binned_ds)

# data_binned.groupby(['variant', 'dataset','bin']).mean()
data = data_binned.groupby(['variant', 'dataset','bin']).mean().score
data
#data_binned['index'] = 10 - data_binned['index']


In [ ]:
dataset = 'gowalla0.3'
models_paths = {}
models_paths['{}_processed_item'.format(dataset)] = {
    'nodis': '/data/prediction/our_model/{}vae_nolambda_few_shot_test_preds_new_new.npz'.format(dataset),
    'BPR' : '/data/prediction/BPR/{}_all_preds_new.npz'.format(dataset),
    'VAE-CF' : "/data/prediction/VAE-CF/{}_all_preds_new.npz".format(dataset),
    'Ours_VAE': '/data/prediction/our_model/{}vae_memory_few_shot_test_preds_new.npz'.format(dataset),
    'Ours_BPR': '/data/prediction/our_model/{}bpr_memory_few_shot_test_preds_new.npz'.format(dataset)
    
}
df_gowalla = get_df_item_recall('gowalla0.3_processed_item', models_paths['gowalla0.3_processed_item'], 'Recall', 50)


In [ ]:
dataset = 'epinions0.3'
models_paths = {}
models_paths['{}_processed_item'.format(dataset)] = {
    'Pronet': '/data/prediction/our_model/{}vae_pronet_few_shot_test_preds_new_new.npz'.format(dataset), 
    'BPR' : '/data/prediction/BPR/{}_all_preds_new.npz'.format(dataset),
    'VAE-CF' : "/data/prediction/VAE-CF/{}_all_preds_new.npz".format(dataset),
    'Ours_VAE': '/data/prediction/our_model/{}vae_memory_few_shot_test_preds_new.npz'.format(dataset),
    'Ours_BPR': '/data/prediction/our_model/{}bpr_memory_few_shot_test_preds_new.npz'.format(dataset),
    "Meta_Rec": '/data/prediction/meta_learning/{}vae_memory_few_shot_test_preds_new.npz'.format(dataset)
    
}
df_epinions = get_df_item_recall('epinions0.3_processed_item', models_paths['epinions0.3_processed_item'], 'Recall', 50)


In [ ]:
# df_epinions['dataset'] = 'epinions'
# df_gowalla['dataset'] = 'gowalla'
df = pd.DataFrame()
df_item = pd.DataFrame()
# df = df.append(df_epinions)
# df = df.append(df_gowalla)

In [ ]:
n_bins= 2
df_epinions['dataset'] = 'epinions'

data_binned = pd.DataFrame()
ds = df_epinions[(df_epinions.dataset == 'epinions') & (df_epinions.item_activity > 0) & (df_epinions.item_activity < 44)]
ds.item_activity *= 0.7
binned_ds = bin_data(ds, 'item_activity', n_bins, 'score', range_min=5, range_max=35)
data = binned_ds
print(binned_ds.groupby('bin')['item'].nunique())
idx = binned_ds.groupby('bin').size().reset_index()['bin'].reset_index()
binned_ds = binned_ds.merge(idx, on='bin', how='inner')
data_binned = data_binned.append(binned_ds)

#df = df.append(data_binned.groupby(['variant', 'dataset','bin']).mean()[['variant', 'dataset', 'score', 'index']])
df = df.append(data_binned)
#data_binned.groupby(['variant', 'dataset','bin']).mean()
data = data_binned.groupby(['variant', 'dataset','bin']).mean().score
data

In [ ]:
items = pd.DataFrame(binned_ds.groupby('bin')['item'].nunique())
idx = items.reset_index()['bin'].reset_index()
items = items.merge(idx, on='bin', how='inner')
items['dataset']='epinions'
items['variant']= 'cdf'
for i in range(1, 5):
    items['item'][i] += items['item'][i - 1]
items['item'] *= 100/9035
df_item = df_item.append(items)

In [ ]:
n_bins= 2
df_yelp['dataset'] = 'yelp'

data_binned = pd.DataFrame()
ds = df_yelp[(df_yelp.dataset == 'yelp') & (df_yelp.item_activity > 10)& (df_yelp.item_activity < 44)]
ds.item_activity *= 0.7
binned_ds = bin_data(ds, 'item_activity', n_bins, 'score', range_min=5, range_max=35)
data = binned_ds
print(binned_ds.groupby('bin')['item'].nunique())
idx = binned_ds.groupby('bin').size().reset_index()['bin'].reset_index()
binned_ds = binned_ds.merge(idx, on='bin', how='inner')
data_binned = data_binned.append(binned_ds)

data_binned.groupby(['variant', 'dataset','bin']).mean()
data = data_binned.groupby(['variant', 'dataset','bin']).mean().score

#data_binned['index'] = 10 - data_binned['index']
df = df.append(data_binned)
#df = df.append(data_binned.groupby(['variant', 'dataset','bin']).mean()[['variant', 'dataset', 'score', 'index']])
data

In [ ]:
items = pd.DataFrame(binned_ds.groupby('bin')['item'].nunique())
idx = items.reset_index()['bin'].reset_index()
items = items.merge(idx, on='bin', how='inner')
items['dataset']='yelp'
items['variant']= 'cdf'
for i in range(1, 5):
    items['item'][i] += items['item'][i - 1]
items['item'] *= 100/10451
df_item = df_item.append(items)

In [ ]:
n_bins= 2
df_weeplaces['dataset'] = 'weeplaces'
data_binned = pd.DataFrame()
ds = df_weeplaces[(df_weeplaces.dataset == 'weeplaces') & (df_weeplaces.item_activity > 10) & (df_weeplaces.item_activity < 44)]
ds.item_activity *= 0.7
binned_ds = bin_data(ds, 'item_activity', n_bins, 'score', range_min=5, range_max=35)
data = binned_ds
print(binned_ds.groupby('bin')['item'].nunique())
idx = binned_ds.groupby('bin').size().reset_index()['bin'].reset_index()
binned_ds = binned_ds.merge(idx, on='bin', how='inner')
data_binned = data_binned.append(binned_ds)

data = data_binned.groupby(['variant', 'dataset','bin']).mean().score

#df = df.append(data_binned.groupby(['variant', 'dataset','bin']).mean())
df = df.append(data_binned)
data
#data_binned.groupby(['variant', 'dataset','bin']).mean()
#data = data_binned.groupby(['variant', 'dataset','bin']).mean().score


In [ ]:
items = pd.DataFrame(binned_ds.groupby('bin')['item'].nunique())
idx = items.reset_index()['bin'].reset_index()
items = items.merge(idx, on='bin', how='inner')
items['dataset']='weeplaces'
items['variant']= 'cdf'
for i in range(1, 5):
    items['item'][i] += items['item'][i - 1]
items['item'] *= 100/11679
df_item = df_item.append(items)

In [ ]:

n_bins= 2
df_gowalla['dataset'] = 'gowalla'
data_binned = pd.DataFrame()
ds = df_gowalla[(df_gowalla.dataset == 'gowalla') & (df_gowalla.item_activity > 10) & (df_gowalla.item_activity < 44)]

ds.item_activity *= 0.7
binned_ds = bin_data(ds, 'item_activity', n_bins, 'score', range_min=5, range_max=35)
data = binned_ds
print(binned_ds.groupby('bin')['item'].nunique())
idx = binned_ds.groupby('bin').size().reset_index()['bin'].reset_index()
binned_ds = binned_ds.merge(idx, on='bin', how='inner')
data_binned = data_binned.append(binned_ds)

#df = df.append(data_binned.groupby(['variant', 'dataset','bin']).mean())
df = df.append(data_binned)

#data_binned.groupby(['variant', 'dataset','bin']).mean()
#data = data_binned.groupby(['variant', 'dataset','bin']).mean().score
data = data_binned.groupby(['variant', 'dataset','bin']).mean().score
data


In [ ]:

items = pd.DataFrame(binned_ds.groupby('bin')['item'].nunique())
idx = items.reset_index()['bin'].reset_index()
items = items.merge(idx, on='bin', how='inner')
items['dataset']='gowalla'
items['variant']= 'cdf'
for i in range(1, 5):
    items['item'][i] += items['item'][i - 1]
items['item'] *= 100/27920
print(items)
df_item = df_item.append(items)

In [ ]:
df_item


In [ ]:
data_binned['index'] = 5 - data_binned['index']

intro_df = data_binned
sns.set(rc={'figure.figsize':(11,6)})
sns.set(font_scale=3)
sns.set_style("ticks")


plot_model_names = ['BPR', 'Ours_BPR', 'VAE-CF', 'Ours_VAE']
data_plot = intro_df[intro_df.variant.isin(plot_model_names)]

pal = sns.mpl_palette("tab10", len(plot_model_names))
pal.reverse()
pal[-1] = sns.color_palette("Blues")[-1]


ax = sns.lineplot(x='index',y='score',hue='variant',data=data_plot, palette=pal,
                hue_order=plot_model_names, linewidth=3, dashes = False)


fontsize=18

ax.set_xlabel("Item groups ordered by decreasing popularity ", fontsize=fontsize)
ax.set_ylabel("Item Recall$@50$", fontsize=fontsize)


# # set subplot title

ax.set_title("", fontsize=fontsize)
#ax.set_yticks(np.arange(0.15, 0.35, 0.1))
# ax.set_yticks(np.arange(0.0, 0.41,  0.01), minor=True)


ax.set_xticks(np.arange(1, 6))

ax.tick_params(direction='in', length=7, width=3, colors='k', which='major', labelsize=15, axis='y')
ax.tick_params(direction='in', length=4, width=2, colors='gray', which='minor', axis='y')

ax.tick_params(direction='in', length=7, width=3, colors='k', which='major', labelsize=15, axis='x')
ax.tick_params(direction='in', length=4, width=2, colors='gray', which='minor', axis='x')


handles, labels = ax.get_legend_handles_labels()
print(handles)
#ax.legend(handles=handles[1:], labels=labels[1:], loc='upper center', bbox_to_anchor=(1, 1), ncol=2, 
#             frameon=False, fontsize=10)
ax.legend(handles=handles[1:], labels=labels[1:],  
             frameon=False, fontsize=fontsize)

sns.despine(offset=20, left=True, trim=True)
# plt.savefig('intro.pdf', bbox_inches='tight')





In [ ]:
df.dataset.unique()
name_map = {'Ours_BPR': 'ProtoCF + BPR', 'Ours_VAE': 'ProtoCF + VAE', 'BPR': 'BPR', 'VAE-CF': 'VAE-CF'}

df['variant'] = df['variant'].apply(lambda x: name_map[x])

In [ ]:
#df.variant.unique(), df.dataset.unique()

# plot_bins = ['(5, 10]', '(10, 15]', "(15, 20]", "(20, 25]", '(25, 30]']
plot_bins = ['5-10', '11-15', "16-20", "21-25", '26-30']

# frequency_bins = [[11, 12, 14, 16, 19, 24, 28, 38, 60, 805],
#                  [12, 15, 19, 24,  32, 40, 50, 68, 108, 759],
#                  [18, 21, 25, 31, 34, 40, 49, 64, 95, 1480],
#                  [12, 14, 16, 19, 24, 28, 33, 42, 66, 2124]]

In [ ]:
df['index'].max()

In [ ]:
sns.set(font_scale=2)
sns.set_style("ticks")
barWidth = 0.25
# df['index'] = df['index'] + 1
# df_item['index'] = df_item['index'] + 1
plot_model_names = ['VAE-CF','BPR','ProtoCF + BPR','ProtoCF + VAE']

# pal = sns.mpl_palette("tab10", len(plot_model_names))
# print(sns.color_palette("Blues"))
# pal.reverse()
# pal[-1] = sns.color_palette("Blues")[-1]
# pal[0] = sns.color_palette("Blues")[-1]
# pal[1] = pal[-2]

pal = sns.mpl_palette("tab10", len(plot_model_names))
pal.reverse()
pal[-1] = sns.color_palette("Blues")[-1]

fig, ax = sns.mpl.pyplot.subplots(1, 4, sharey=False, figsize=(36, 6))

fig.subplots_adjust(wspace=0.05, hspace=5)

ax2 = ax[0].twinx()
# ax2= sns.lineplot(data=df_item[df_item.dataset == 'epinions'], x = 'index', y='item')
g2 = ax2.bar(np.arange(5) , df_item[df_item.dataset == 'epinions'].item, color='lightblue', alpha=0.5,
             width=barWidth, edgecolor='white')


ax2.set_yticks(np.arange(0, 101, 50))
ax2.set_yticks(np.arange(0, 101, 10), minor=True)
ax2.set_yticklabels(map(lambda x : x + "%", map(str, np.arange(0, 101, 50))))

ax[0] = sns.lineplot(x='index',y='score',hue='variant', style='variant', size='variant', data=df[df.dataset=="epinions"], 
                  palette=pal, ax = ax[0], hue_order=plot_model_names, size_order=plot_model_names,
                     style_order=plot_model_names, 
                     dashes = [ (4, 1.5), (4, 1.5), (1, 0) ,  (1, 0),  ""],
                    sizes=[1.5,1.5,2.5,2.5])


ax[1] = sns.lineplot(x='index',y='score',hue='variant', style='variant',  size='variant', data=df[df.dataset=="yelp"], 
                  palette=pal, ax = ax[1], hue_order=plot_model_names, style_order=plot_model_names, size_order=plot_model_names,
             legend=False, 
                     dashes = [ (4, 1.5), (4, 1.5), (1, 0) ,  (1, 0),  ""],
                    sizes=[1.5,1.5,2.5,2.5])



ax[2] = sns.lineplot(x='index',y='score',hue='variant', style='variant', size='variant', data=df[df.dataset=="weeplaces"], 
                  palette=pal,  ax = ax[2], hue_order=plot_model_names,style_order=plot_model_names, size_order=plot_model_names,
             legend=False,                     dashes = [ (4, 1.5), (4, 1.5), (1, 0) ,  (1, 0),  ""],
                    sizes=[1.5,1.5,2.5,2.5])

ax[3] = sns.lineplot(x='index',y='score',hue='variant', style='variant', size='variant', data=df[df.dataset=="gowalla"], 
                  palette=pal, ax = ax[3], hue_order=plot_model_names, style_order=plot_model_names, size_order=plot_model_names,
             legend=False,                     dashes = [ (4, 1.5), (4, 1.5), (1, 0) ,  (1, 0),  ""],
                    sizes=[1.5,1.5,2.5,2.5])

fontsize=28


ax[0].set_yticks(np.arange(0.05, 0.201, 0.05))
ax[0].set_yticks(np.arange(0.0, 0.201, 0.01), minor=True)



ax[1].set_yticks(np.arange(0.05, 0.151, 0.05))
ax[1].set_yticks(np.arange(0.0, 0.151, 0.01), minor=True)
ax4 = ax[1].twinx()
# ax4= sns.lineplot(data=df_item[df_item.dataset == 'yelp'], x = 'index', y='item
g2 = ax4.bar(np.arange(5) , df_item[df_item.dataset == 'yelp'].item, color='lightblue', alpha=0.5,
             width=barWidth, edgecolor='white')
ax4.set_yticks(np.arange(0, 101, 50))
ax4.set_yticks(np.arange(0, 101, 10), minor=True)
ax4.set_yticklabels(map(lambda x : x + "%", map(str, np.arange(0, 101, 50))))
ax[2].set_yticks(np.arange(0.05, 0.301, 0.05))
ax[2].set_yticks(np.arange(0.05, 0.301, 0.01), minor=True)
ax6 = ax[2].twinx()

# ax6= sns.lineplot(data=df_item[df_item.dataset == 'weeplaces'], x = 'index', y='item')
g2 = ax6.bar(np.arange(5) , df_item[df_item.dataset == 'weeplaces'].item, color='lightblue', alpha=0.5,
             width=barWidth, edgecolor='white')
ax6.set_yticks(np.arange(0, 101, 50))
ax6.set_yticks(np.arange(0, 101, 10), minor=True)
ax6.set_yticklabels(map(lambda x : x + "%", map(str, np.arange(0, 101, 50))))
ax[3].set_yticks(np.arange(0.1, 0.251, 0.05))
ax[3].set_yticks(np.arange(0.1, 0.251, 0.01), minor=True)
ax8 = ax[3].twinx()
# ax8= sns.lineplot(data=df_item[df_item.dataset == 'gowalla'], x = 'index', y='item')
g2 = ax8.bar(np.arange(5), df_item[df_item.dataset == 'gowalla'].item, color='lightblue', alpha=0.5,
             width=barWidth, edgecolor='white')
ax8.set_yticks(np.arange(0, 101, 50))
ax8.set_yticks(np.arange(0, 101, 10), minor=True)
ax8.set_yticklabels(map(lambda x : x + "%", map(str, np.arange(0, 101, 50))))
#ax[0].set_ylim([0.05, 0.201])
#ax[1].set_ylim([0.05, 0.201])


for idx in range(0, 4):
    ax[idx].set_xlabel("Training interactions per item", fontsize=fontsize)
    #ax[idx].set_ylabel("Recall$@K$", fontsize=fontsize)
    ax[idx].set_ylabel('')
ax[0].set_ylabel("Item Recall$@50$", fontsize=fontsize)

dataset_list = ['Epinions', 'Yelp', 'Weeplaces', 'Gowalla']
for it in range(0, 4):
    
    ax[it].set_xticks(np.arange(0, 5))
    ax[it].set_xticklabels(plot_bins)

    ax[it].tick_params(direction='in', length=6, width=2, colors='k', which='major', axis='both')
    ax[it].tick_params(direction='in', length=4, width=1, colors='gray', which='minor', axis='both')
    
    ax[it].tick_params(labelsize=20)
    ax[it].legend()
    ax[it].set_title(dataset_list[it])

sns.despine(offset=10, left=True, trim=True)
sns.mpl.pyplot.tight_layout()

handles, labels = ax[0].get_legend_handles_labels()
print(handles)
ax[0].legend(handles=handles[1:], labels=labels[1:], loc='upper center', bbox_to_anchor=(2.2, 1.35), ncol=7, frameon=False, fontsize=30)

ax[1].legend(frameon=False)
ax[2].legend(frameon=False)
ax[3].legend(frameon=False)

ax8.set_ylabel("Cumulative fraction of items", fontsize=fontsize)

plt.savefig('item_recall_counts.pdf', bbox_inches='tight')

plt.show()

In [ ]:
plot_bins[0]

In [ ]:
sns.set(font_scale=2)
sns.set_style("ticks")

# df['index'] = 10 - df['index']
df['index'] = 5 - df['index']

plot_model_names = ['VAE-CF','BPR','ProtoCF + BPR','ProtoCF + VAE']

# pal = sns.mpl_palette("tab10", len(plot_model_names))
# print(sns.color_palette("Blues"))
# pal.reverse()
# pal[-1] = sns.color_palette("Blues")[-1]
# pal[0] = sns.color_palette("Blues")[-1]
# pal[1] = pal[-2]

pal = sns.mpl_palette("tab10", len(plot_model_names))
pal.reverse()
pal[-1] = sns.color_palette("Blues")[-1]

fig, ax = sns.mpl.pyplot.subplots(1, 4, sharey=False, figsize=(36, 6))

fig.subplots_adjust(wspace=0.05, hspace=5)

ax[0] = sns.lineplot(x='index',y='score',hue='variant', style='variant', size='variant', data=df[df.dataset=="epinions"], 
                  palette=pal, ax = ax[0], hue_order=plot_model_names, size_order=plot_model_names,
                     style_order=plot_model_names, 
                     dashes = [ (4, 1.5), (4, 1.5), (1, 0) ,  (1, 0),  ""],
                    sizes=[1.5,1.5,2.5,2.5])


ax[1] = sns.lineplot(x='index',y='score',hue='variant', style='variant',  size='variant', data=df[df.dataset=="yelp"], 
                  palette=pal, ax = ax[1], hue_order=plot_model_names, style_order=plot_model_names, size_order=plot_model_names,
             legend=False, 
                     dashes = [ (4, 1.5), (4, 1.5), (1, 0) ,  (1, 0),  ""],
                    sizes=[1.5,1.5,2.5,2.5])
ax[2] = sns.lineplot(x='index',y='score',hue='variant', style='variant', size='variant', data=df[df.dataset=="weeplaces"], 
                  palette=pal,  ax = ax[2], hue_order=plot_model_names,style_order=plot_model_names, size_order=plot_model_names,
             legend=False,                     dashes = [ (4, 1.5), (4, 1.5), (1, 0) ,  (1, 0),  ""],
                    sizes=[1.5,1.5,2.5,2.5])

ax[3] = sns.lineplot(x='index',y='score',hue='variant', style='variant', size='variant', data=df[df.dataset=="gowalla"], 
                  palette=pal, ax = ax[3], hue_order=plot_model_names, style_order=plot_model_names, size_order=plot_model_names,
             legend=False,                     dashes = [ (4, 1.5), (4, 1.5), (1, 0) ,  (1, 0),  ""],
                    sizes=[1.5,1.5,2.5,2.5])

fontsize=25


ax[0].set_yticks(np.arange(0.05, 0.301, 0.05))
ax[0].set_yticks(np.arange(0.0, 0.301, 0.01), minor=True)

ax[1].set_yticks(np.arange(0.05, 0.301, 0.05))
ax[1].set_yticks(np.arange(0.0, 0.301, 0.01), minor=True)

ax[2].set_yticks(np.arange(0.05, 0.301, 0.05))
ax[2].set_yticks(np.arange(0.05, 0.301, 0.01), minor=True)

ax[3].set_yticks(np.arange(0.1, 0.301, 0.05))
ax[3].set_yticks(np.arange(0.1, 0.301, 0.01), minor=True)

#ax[0].set_ylim([0.05, 0.201])
#ax[1].set_ylim([0.05, 0.201])


for idx in range(0, 4):
    ax[idx].set_xlabel("Item groups by decreasing frequency", fontsize=fontsize)
    #ax[idx].set_ylabel("Recall$@K$", fontsize=fontsize)
    ax[idx].set_ylabel('')
ax[0].set_ylabel("Item Recall$@50$", fontsize=fontsize)

dataset_list = ['Epinions', 'Yelp', 'Weeplaces', 'Gowalla']
for it in range(0, 4):
    #ax[it].set_xticks(np.arange(1, 12, 2))
    #ax[it].set_xticks(np.arange(1, 11, 1), minor=True)
    # ax[it].set_xlim((1,11))

    ax[it].set_xticks(np.arange(1, 11))    
    # ax[it].set_xticklabels(list(map(str, range(5, 30, 5))))
    ax[it].tick_params(direction='in', length=6, width=2, colors='k', which='major', axis='both')
    ax[it].tick_params(direction='in', length=4, width=1, colors='gray', which='minor', axis='both')
    
    ax[it].tick_params(labelsize=15)
    ax[it].legend()
    ax[it].set_title(dataset_list[it])

sns.despine(offset=10, left=True, trim=True)
sns.mpl.pyplot.tight_layout()

handles, labels = ax[0].get_legend_handles_labels()
print(handles)
ax[0].legend(handles=handles[1:], labels=labels[1:], loc='upper center', bbox_to_anchor=(2.2, 1.35), ncol=7, frameon=False, fontsize=30)

ax[1].legend(frameon=False)
ax[2].legend(frameon=False)
ax[3].legend(frameon=False)
#plt.savefig('item_recall.pdf', bbox_inches='tight')

plt.show()

In [ ]:



data_binned['index'] = 10 - data_binned['index']

intro_df = data_binned
sns.set(rc={'figure.figsize':(11,6)})
sns.set(font_scale=3)
sns.set_style("ticks")


plot_model_names = ['BPR', 'Ours_BPR', 'VAE-CF', 'Ours_VAE']
data_plot = intro_df[intro_df.variant.isin(plot_model_names)]

pal = sns.mpl_palette("tab10", len(plot_model_names))
pal.reverse()
pal[-1] = sns.color_palette("Blues")[-1]


ax = sns.lineplot(x='index',y='score',hue='variant',data=data_plot, palette=pal,
                hue_order=plot_model_names, linewidth=3, dashes = False)


fontsize=18

ax.set_xlabel("Item groups ordered by decreasing popularity ", fontsize=fontsize)
ax.set_ylabel("Item Recall$@50$", fontsize=fontsize)


# # set subplot title

ax.set_title("", fontsize=fontsize)
#ax.set_yticks(np.arange(0.15, 0.35, 0.1))
# ax.set_yticks(np.arange(0.0, 0.41,  0.01), minor=True)


ax.set_xticks(np.arange(1, 11))

ax.tick_params(direction='in', length=7, width=3, colors='k', which='major', labelsize=15, axis='y')
ax.tick_params(direction='in', length=4, width=2, colors='gray', which='minor', axis='y')

ax.tick_params(direction='in', length=7, width=3, colors='k', which='major', labelsize=15, axis='x')
ax.tick_params(direction='in', length=4, width=2, colors='gray', which='minor', axis='x')


handles, labels = ax.get_legend_handles_labels()
print(handles)
#ax.legend(handles=handles[1:], labels=labels[1:], loc='upper center', bbox_to_anchor=(1, 1), ncol=2, 
#             frameon=False, fontsize=10)
ax.legend(handles=handles[1:], labels=labels[1:],  
             frameon=False, fontsize=fontsize)

sns.despine(offset=20, left=True, trim=True)
# plt.savefig('intro.pdf', bbox_inches='tight')










In [ ]:
df_popular, df_fs = get_df_item_recall(dataset, models_paths[dataset], 'Recall', 50 , mode='popular')

In [ ]:
df_popular.mean()

In [ ]:
df_fs.mean()

In [ ]:
n_bins= 10
df_fs['dataset'] = dataset
df_popular['dataset'] = dataset

data_binned = pd.DataFrame()
    
# ds = df_fs[(df_fs.dataset == dataset) ]
ds = df_popular[(df_popular.dataset == dataset) ]
# ds = ds.append(df_popular[(df_popular.dataset == dataset) ])

binned_ds = bin_data(ds, 'item_activity', n_bins, 'score')
data = binned_ds
idx = binned_ds.groupby('bin').size().reset_index()['bin'].reset_index()
binned_ds = binned_ds.merge(idx, on='bin', how='inner')
data_binned = data_binned.append(binned_ds)

data_binned.groupby(['variant', 'dataset','bin']).mean()
data = data_binned.groupby(['variant', 'dataset','bin']).mean().score
data_binned.to_pickle("dataframe/intro.pkl")

In [ ]:
data

In [ ]:
our_BPR = data[:10].to_list()
our_VAE = data[10:20].to_list()
VAE = data[20:].to_list()

In [ ]:
bins = [1,2,3,4,5,6,7,8,9,10][::-1]
import matplotlib.pyplot as plt
plt.plot(bins, our_BPR, label="our_BPR")
plt.plot(bins, our_VAE, label="our_VAE")
plt.plot(bins, VAE, label="VAE")
# plt.plot(bins, DAE, label="DAE")
plt.xlabel("Item Activity from Most Active to Least")
plt.ylabel("Recall 50")
plt.legend()
# dict_1 = {'dataset':'epinions0.5', 'variant': 'VAE', 'bins':bins, 'scores': VAE}
# dict_2 = {'dataset':'epinions0.5', 'variant': 'DAE', 'bins':bins, 'scores': DAE}
# temp_df = pd.DataFrame([dict_1, dict_2])
# print(temp_df)
# df_plot = pandas_explode(temp_df, 'bins', cols={'scores':temp_df['scores']})
# df_plot

In [ ]:
print("FEWSHOT")
print(df_fs[df_fs.variant == 'VAE-CF'].mean())
print(df_fs[df_fs.variant == 'BPR'].mean())
print("POPULAR")
print(df_popular[df_popular.variant == 'VAE-CF'].mean())
print(df_popular[df_popular.variant == 'BPR'].mean())

In [ ]:
import time
dataset = 'gowalla'
datasets = ['{}_processed_item'.format(dataset)]

# Static best model output paths for each dataset.

models_paths = {}
models_paths['{}_processed_item'.format(dataset)] = {
    'BPR' : '/data/prediction/BPR/{}_all_preds.npz'.format(dataset),
    'VAE-CF' : "/data/prediction/VAE-CF/{}_all_preds.npz".format(dataset),
    'DAE' : "/data/prediction/DAE/{}_all_preds.npz".format(dataset)
    # 'Ours': '/data/prediction/our_model/{}_memory_few_shot_test_preds.npz'.format(dataset)
}

# Define best model paths here for each dataset (has to be hardcoded).

df = pd.DataFrame()
for ds in datasets:
    print (ds)
    df_ds = get_df_all(ds, models_paths[ds],'Recall', 50)
    df = df.append(df_ds)



In [ ]:
df[df.variant == 'Ours'].mean()

In [ ]:
n_bins=4

data_binned = pd.DataFrame()

for ds_name in datasets:
    
    ds = df[(df.dataset == ds_name) ]
    print(ds)
    binned_ds = bin_data(ds, 'user_activity', 4, 'score')
    print(binned_ds)
    idx = binned_ds.groupby('bin').size().reset_index()['bin'].reset_index()
    binned_ds = binned_ds.merge(idx, on='bin', how='inner')
    data_binned = data_binned.append(binned_ds)

data_binned.groupby(['variant', 'dataset','bin']).mean()



In [ ]:
plot_model_names = ['BPR', 'VAE-CF', 'Ours']

sns.set(font_scale=1)
sns.set_style("ticks")
flatui= ["#1f78b4","#33a02c","#6a3d9a","#fb8072","#8dd3c7", "#fdb462","#fb9a99", "#fb9a99", "#fb9a99"]

hue_kws = dict(
               marker=["o", "^", "v", "s", "p", "*", ".", ".", "."],
               ls=["--", "--", "-.", "-.", "-", "-.", "-.", "-.", "-."],
               linewidth=[5, 5, 5, 5, 7, 4, 4, 4, 4])

pal = sns.mpl_palette("tab10", len(plot_model_names))
pal.reverse()
pal[-1] = sns.color_palette("Blues")[-1]

g = sns.catplot(kind='bar', x="index", y="score", hue="variant", data=data_binned,
               col="dataset", col_wrap=1, aspect=1.5,height=11, sharex=False,
                palette=pal, hue_order=plot_model_names, legend=False, sharey=False)

fontsize=40

g.set_xlabels("User Activity Quartiles", fontsize=fontsize)
g.set_ylabels("NDCG$@50$", fontsize=fontsize)

g.axes[0].set_title("Dataset name", fontsize=50)


for it in range(0, 1):
    g.axes[it].tick_params(direction='in', length=12, width=6, colors='k', which='major', labelsize=35, axis='y')
    g.axes[it].tick_params(direction='in', length=6, width=3, colors='gray', which='minor', axis='y')
        
#g.axes[0].set_yticks(np.arange(0.0, 0.31, 0.1))
#g.axes[0].set_yticks(np.arange(0.0, 0.31, 0.02), minor=True)

g.axes[0].set_xticklabels(map(lambda x : "Q"+str(x), range(1, 5)), fontsize=30)

g.despine(offset=10, left=True, trim=True, bottom=True)
plt.subplots_adjust(hspace=.4)

plt.legend(bbox_to_anchor=(-0.15, 0.05, 1.0, 1.2),ncol=len(plot_model_names), fontsize=50, frameon=False)

g.axes[0].tick_params(labelsize=40)

# g.savefig("user_quartile.pdf")
plt.show()


